# SQL and Pandas Coding Exercises with Answers

## Introduction

This notebook contains a series of coding exercises focused on data manipulation using Pandas, designed to enhance your skills in working with tabular data. Each exercise tackles a specific question and provides a structured Pandas-based solution.

## Exercise Questions

### 1. How many total customers does each employee hired after 2014 have?

This exercise involves calculating the number of unique customers associated with each employee hired after the year 2014. It requires merging employee and order data, filtering based on hire dates, and grouping by employee to count distinct customers.

### 2. What is the total purchase amount by American, French, and German customers after 2007?

Here, the goal is to compute the total purchase amount made by customers from specific countries (USA, France, Germany) after the year 2007. This requires merging order and order details data, filtering by country and date, and computing the total sales amount considering quantities, unit prices, and discounts.

### 3. Which countries have made purchases exceeding 40,000 USD?

This question focuses on identifying countries where total purchases exceed $40,000 USD. It involves merging order and order details data, calculating total sales amounts per country, and filtering to find countries with total purchases above the specified threshold.

### 4. What are the average, count, and sum of sales for each customer who purchased beverage and seafood products, and had them exported by shippers #2 and #3 to the US?

This exercise computes average, count, and sum of sales for customers who purchased specific product categories (beverages and seafood), and used specified shippers to export to the US. It requires merging multiple datasets, filtering by product categories, shippers, and destination country, and aggregating sales statistics per customer.

### 5. What is the latest order date for each customer?

Here, the task is to find the latest order date for each customer. This involves grouping order data by customer and identifying the maximum order date per customer.

### 6. What is the sales ranking of each customer? Please mention the ranking in a separate column.

The final exercise calculates the sales ranking for each customer based on their total purchase amount. It requires aggregating sales data per customer, ranking customers based on their sales amounts in descending order, and assigning a rank to each customer.

## Usage

Each exercise provides a Pandas-based solution to its respective question. The provided example codes demonstrate how to load, merge, filter, manipulate, and aggregate data using Pandas DataFrame operations. These exercises are designed to help you practice and improve your data manipulation skills in Python.

Feel free to explore and modify the provided Pandas code snippets to deepen your understanding and adapt them to different datasets or scenarios.


In [15]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Exercise Questions

### 1. How many total customers does each employee hired after 2014 have?

In [29]:
# Load HR_Employees and Sales_Orders datasets
HR_Employees = pd.read_csv('HR_Employees.csv')
Sales_Orders = pd.read_csv('Sales_Orders.csv')

# Convert hiredate column to datetime in HR_Employees
HR_Employees[['hiredate', 'birthdate']] = HR_Employees[['hiredate', 'birthdate']].apply(pd.to_datetime)

# Merge HR_Employees and Sales_Orders on 'empid'
Employee_Customer_Merge = pd.merge(HR_Employees, Sales_Orders, how='inner', on='empid')

# Filter employees hired after 2014
Employees_Hired_After_2014 = Employee_Customer_Merge.loc[Employee_Customer_Merge['hiredate'].dt.year >= 2014]

# Count number of customers for each employee
Total_Customers_Per_Employee = Employees_Hired_After_2014.groupby(['firstname', 'lastname', 'empid']).count()['custid'].sort_values(ascending=False).reset_index().rename(columns={'custid': 'Total_Number'})

Total_Customers_Per_Employee

,firstname,lastname,empid,Total_Number
0,Yael,Peled,4,156
1,Maria,Cameron,8,104
2,Russell,King,7,72
3,Paul,Suurs,6,67
4,Patricia,Doyle,9,43
5,Sven,Mortensen,5,42


### 2- What is the total purchase amount by American, French, and German customers after 2007?


In [31]:
# Load Sales_Orders and Sales_OrderDetails datasets
Sales_Orders = pd.read_csv('Sales_Orders.csv')
Sales_OrderDetails = pd.read_csv('Sales_OrderDetails.csv')

# Convert orderdate column to datetime in Sales_Orders
Sales_Orders['orderdate'] = pd.to_datetime(Sales_Orders['orderdate'])

# Merge Sales_Orders and Sales_OrderDetails on 'orderid'
Order_OrderDetails_Merge = pd.merge(Sales_Orders, Sales_OrderDetails, on='orderid')

# Filter orders by shipcountry and orderdate
Filtered_Orders = Order_OrderDetails_Merge.loc[(Order_OrderDetails_Merge['shipcountry'].isin(['France', 'USA', 'Germany'])) & (Order_OrderDetails_Merge['orderdate'].dt.year >= 2007)]

# Calculate total sales amount for each product
Filtered_Orders['Total_Sales_Amount'] = Filtered_Orders['qty'] * Filtered_Orders['unitprice'] * (1 - Filtered_Orders['discount'])

# Calculate total purchase amount
Total_Purchase_Amount = Filtered_Orders['Total_Sales_Amount'].sum()

Total_Purchase_Amount

557227.5665000001

### 3- Which countries have made purchases exceeding 40,000 USD?


In [32]:
# Copy Order_OrderDetails_Merge dataframe
Copy_Merge = Order_OrderDetails_Merge.copy()

# Calculate total sales amount for each country
Copy_Merge['Total_Sales_Amount'] = Order_OrderDetails_Merge['qty'] * Order_OrderDetails_Merge['unitprice'] * (1 - Order_OrderDetails_Merge['discount'])
Country_Total_Amount = Copy_Merge.groupby('shipcountry').sum()['Total_Sales_Amount'].sort_values(ascending=False).to_frame().reset_index().rename(columns={'Total_Sales_Amount': 'Total_Amount'})

# Filter countries with purchases exceeding 40,000 USD
High_Spending_Countries = Country_Total_Amount.loc[Country_Total_Amount['Total_Amount'] >= 40000]

High_Spending_Countries

,shipcountry,Total_Amount
0,USA,245584.6105
1,Germany,230284.6335
2,Austria,128003.8385
3,Brazil,106925.7765
4,France,81358.3225
5,UK,58971.3100
6,Venezuela,56810.6290
7,Sweden,54495.1400
8,Canada,50196.2900
9,Ireland,49979.9050


### 4- What are the average, count, and sum of sales for each customer who purchased beverage and seafood products, and had them exported by shippers #2 and #3 to the US?

In [33]:
# Load Production_Categories, Production_Products, Production_Suppliers, Sales_OrderDetails, and Sales_Orders datasets
Production_Categories = pd.read_csv('Production_Categories.csv')
Production_Products = pd.read_csv('Production_Products.csv')
Production_Suppliers = pd.read_csv('Production_Suppliers.csv')
Sales_OrderDetails = pd.read_csv('Sales_OrderDetails.csv')
Sales_Orders = pd.read_csv('Sales_Orders.csv')

# Merge necessary tables to get required data
Merge4 = pd.merge(Production_Categories, Production_Products, on='categoryid').merge(Production_Suppliers, on='supplierid').merge(Sales_OrderDetails, on='productid').merge(Sales_Orders, on='orderid')

# Calculate sale amount for each product
Merge4['Total_Sales_Amount'] = Merge4['qty'] * Merge4['unitprice_x'] * (1 - Merge4['discount'])

# Filter data by specific conditions
Filtered_Data = Merge4.loc[(Merge4['shipcountry'] == 'USA') & (Merge4['categoryid'].isin([1, 8])) & (Merge4['shipperid'].isin([2, 3]))]

# Calculate average, count, and sum of sales for each customer
Customer_Sales_Stats = Filtered_Data.groupby(['custid', 'categoryname']).agg({'Total_Sales_Amount': ['mean', 'sum', 'count']}).reset_index()

Customer_Sales_Stats

custid categoryname Total_Sales_Amount                  
                                     mean         sum count
0      32    Beverages        2338.875000  11694.3750     5
1      32      Seafood         456.000000    456.0000     1
2      36      Seafood          60.000000     60.0000     1
3      43      Seafood         184.000000    184.0000     1
4      45    Beverages         162.750000    162.7500     1
5      45      Seafood         310.376250    620.7525     2
6      48    Beverages         108.000000    216.0000     2
7      48      Seafood         625.000000    625.0000     1
8      55    Beverages         785.000000   2355.0000     3
9      55      Seafood         730.700000   2192.1000     3
10     65    Beverages        2017.625000  20176.2500    10
11     65      Seafood          86.168571    603.1800     7
12     71    Beverages         614.825000   7377.9000    12
13     71      Seafood         715.800000   9305.4000    13
14     75    Beverages        5006.500000   5006.5000     1
15     75      Seafood         516.921333   1550.7640     3
16     77    Beverages        1387.500000   2775.0000     2
17     77      Seafood          95.000000     95.0000     1
18     78    Beverages          72.000000     72.0000     1
19     78      Seafood         184.000000    184.0000     1
20     82    Beverages         141.666667    425.0000     3
21     82      Seafood          36.800000     36.8000     1
22     89    Beverages        1655.000000   8275.0000     5
23     89      Seafood         470.833333   2825.0000     6

### 5- What is the latest order date for each customer?

In [34]:
# Find latest order date for each customer
Latest_Order_Date = Sales_Orders.groupby('custid')['orderdate'].max().reset_index()

Latest_Order_Date

,custid,orderdate
0,1,2016-04-09
1,2,2016-03-04
2,3,2016-01-28
3,4,2016-04-10
4,5,2016-03-04
...,...,...
84,87,2016-04-15
85,88,2016-03-09
86,89,2016-05-01
87,90,2016-04-07


### 6- What is the sales ranking of each customer? Please mention the ranking in a separate column.

In [35]:
# Copy Sales_Orders and Sales_OrderDetails datasets
Copy_Merge = pd.merge(Sales_Orders, Sales_OrderDetails, on='orderid')

# Calculate total sales amount for each customer
Copy_Merge['Total_Sales_Amount'] = Copy_Merge['qty'] * Copy_Merge['unitprice'] * (1 - Copy_Merge['discount'])
Total_Sales_Amount = Copy_Merge.groupby('custid')['Total_Sales_Amount'].sum().to_frame().reset_index()

# Add ranking based on sales amount
Total_Sales_Amount['Ranking'] = Total_Sales_Amount['Total_Sales_Amount'].rank(ascending=False).astype(int)

Total_Sales_Amount

,custid,Total_Sales_Amount,Ranking
0,1,4273.0000,57
1,2,1402.9500,84
2,3,7023.9775,46
3,4,13390.6500,31
4,5,24927.5775,13
...,...,...,...
84,87,15648.7025,28
85,88,6068.2000,52
86,89,27363.6050,10
87,90,3161.3500,69
